In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
from rtb.datasets import get_dataset
from rtb.utils import rolling_window_sampler, one_window_sampler

In [4]:
ds = get_dataset(
    name="rtb-product",
    root="../../data",
    # default is books-5-core
    # category="fashion",
    # use_5_core=False,
    # process=True,
)

loading table ../../data/rtb-product/books-5-core/processed/db/customer.parquet...
done in 0.57 seconds.
loading table ../../data/rtb-product/books-5-core/processed/db/product.parquet...
done in 1.78 seconds.
loading table ../../data/rtb-product/books-5-core/processed/db/review.parquet...
done in 43.70 seconds.


In [5]:
db = ds._db

In [6]:
customer = db.tables["customer"].df
product = db.tables["product"].df
review = db.tables["review"].df

In [7]:
db.tables["customer"].df

,customer_id,customer_name
0,0,TW Ervin II
1,1,Amazon Customer
2,2,Tekla Borner
3,3,Deborah K Woroniecki
4,4,E
...,...,...
1850188,1850188,Amazon Customer
1850189,1850189,AmizoP
1850190,1850190,JuicyBeans
1850191,1850191,Halley Cortez


In [8]:
product

,product_id,category,brand,title,description,price
0,0,"[Books, Literature & Fiction, Dramas & Plays]",Visit Amazon's William Shakespeare Page,Love's Labour's Lost: Performed by Derek Jacob...,William Shakespeare is widely regarded as the ...,20.93
1,1,"[Books, Children's Books, Literature & Fiction]",Louis (ed) UNTERMEYER,The golden treasury of poetry,Collection of Poetry,23.98
2,2,"[Books, Mystery, Thriller & Suspense]",Visit Amazon's Agatha Christie Page,The Blue Geranium (Miss Marple),None,8.82
3,3,"[Books, Humor & Entertainment]",Visit Amazon's Tom Baker Page,Who on Earth Is Tom Baker?,Tom Baker's autobiography covers his childhood...,159.95
4,4,"[Books, Christian Books & Bibles, Bible Study ...",Visit Amazon's Clarence Larkin Page,The Greatest Book on &quot;Dispensational Trut...,The Greatest Book on Dispensational Truth in t...,44.76
...,...,...,...,...,...,...
506201,506201,None,"Tricia & Putman, Cara Goyer",Mason Jar Mayhem,"Spring has come to Sugarcreek, and the communi...",15.05
506202,506202,None,Author,The Little Book of Pussy (2013-06-16) [Paperback],From the time The Big Penis Book was published...,17.00
506203,506203,None,,The Age of Not Believing: A Christmas Tale by ...,None,42.93
506204,506204,None,,The Charlies Angels' Casebook,None,17.87


In [9]:
review

,review_time,rating,verified,review_text,summary,customer_id,product_id
0,2005-03-30,5.0,False,"The King, the Mice and the Cheese by Nancy Gur...",A story children will love and learn from,0,14
1,2016-06-20,5.0,True,The kids loved it!,Five Stars,1,14
2,2016-01-24,5.0,True,My students (3 & 4 year olds) loved this book!...,Five Stars,2,14
3,2015-07-09,5.0,False,LOVE IT,Five Stars,3,14
4,2015-01-18,5.0,True,Great!,Five Stars,4,14
...,...,...,...,...,...,...,...
21940283,2015-02-01,5.0,False,"Testimonials\n\nPerhaps once in a decade, or e...",a book like YOUR POWER GIVES YOU THE WINGS TO ...,1850154,337213
21940284,2015-01-25,5.0,False,"Testimonials\nWhen I need to dig deeper, push ...","Coman has accomplished amazing things, but it ...",1850155,337213
21940285,2014-10-09,5.0,False,Best book in Feng Shui and I recommend on buyi...,"Feng-Shui - The Art to Succeed, Conquer and Ov...",1850156,337213
21940286,2014-10-04,5.0,False,MASTER JUDO AND FENG SHUI DUMITRU D.COMAN Wit...,MASTER JUDO AND FENG SHUI DUMITRU D. COMAN Wit...,1850157,337213


## LTV

In [27]:
window_size = ds.tasks["ltv"].window_sizes[0]
window_size

Timedelta('364 days 00:00:00')

### train table

In [14]:
# default time window sampler used inside make_train_table
time_window_df = rolling_window_sampler(
    ds.min_time, ds.train_max_time, window_size, window_size
)
time_window_df

,window_min_time,window_max_time
0,1996-06-25 00:00:01,1997-06-24 00:00:01
1,1997-06-24 00:00:01,1998-06-23 00:00:01
2,1998-06-23 00:00:01,1999-06-22 00:00:01
3,1999-06-22 00:00:01,2000-06-20 00:00:01
4,2000-06-20 00:00:01,2001-06-19 00:00:01
5,2001-06-19 00:00:01,2002-06-18 00:00:01
6,2002-06-18 00:00:01,2003-06-17 00:00:01
7,2003-06-17 00:00:01,2004-06-15 00:00:01
8,2004-06-15 00:00:01,2005-06-14 00:00:01
9,2005-06-14 00:00:01,2006-06-13 00:00:01


In [16]:
train_table = ds.make_train_table("ltv", window_size)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [18]:
# here count is the number of products which contributed to the sum aggregation in ltv
train_table.df

,window_min_time,window_max_time,customer_id,ltv,count
0,1996-06-25 00:00:01,1997-06-24 00:00:01,1843200,0.00,0
1,1996-06-25 00:00:01,1997-06-24 00:00:01,1843201,0.00,0
2,1996-06-25 00:00:01,1997-06-24 00:00:01,1843203,0.00,0
3,1996-06-25 00:00:01,1997-06-24 00:00:01,1843205,0.00,0
4,1996-06-25 00:00:01,1997-06-24 00:00:01,1843206,0.00,0
...,...,...,...,...,...
31453276,2012-06-05 00:00:01,2013-06-04 00:00:01,306154,24.82,2
31453277,2012-06-05 00:00:01,2013-06-04 00:00:01,306262,30.35,2
31453278,2012-06-05 00:00:01,2013-06-04 00:00:01,307022,0.00,0
31453279,2012-06-05 00:00:01,2013-06-04 00:00:01,307037,0.00,0


In [19]:
# frequency of counts in percent
train_table.df["count"].value_counts(normalize=True) * 100

count
0      95.140863
1       2.509268
2       0.923754
3       0.471935
4       0.286215
         ...    
433     0.000003
295     0.000003
770     0.000003
701     0.000003
339     0.000003
Name: proportion, Length: 335, dtype: float64

_Note: this is very skewed! Test table is not this skewed (95% 0-counts vs 47%)._

## val table

In [40]:
# time sampler used inside make_val_table
time_window_df = one_window_sampler(ds.val_max_time, window_size)
time_window_df

,window_min_time,window_max_time
0,2016-07-07 00:00:01,2017-07-06


In [41]:
val_table = ds.make_val_table("ltv", window_size)

In [42]:
val_table.df

,window_min_time,window_max_time,customer_id,ltv,count
0,2014-04-16 00:00:01,2015-04-15,1843201,0.00,0
1,2014-04-16 00:00:01,2015-04-15,1843202,0.00,0
2,2014-04-16 00:00:01,2015-04-15,1843203,0.00,0
3,2014-04-16 00:00:01,2015-04-15,1843204,0.00,0
4,2014-04-16 00:00:01,2015-04-15,1843205,0.00,0
...,...,...,...,...,...
1850188,2014-04-16 00:00:01,2015-04-15,1636118,21.88,2
1850189,2014-04-16 00:00:01,2015-04-15,1636647,0.00,0
1850190,2014-04-16 00:00:01,2015-04-15,1636780,0.00,0
1850191,2014-04-16 00:00:01,2015-04-15,1637589,0.00,0


In [43]:
val_table.df["count"].value_counts(normalize=True) * 100

count
0       47.376679
1       17.256470
2       11.010797
3        7.313994
4        4.936296
          ...    
1171     0.000054
232      0.000054
228      0.000054
284      0.000054
304      0.000054
Name: proportion, Length: 303, dtype: float64

## Churn

In [26]:
window_size = ds.tasks["churn"].window_sizes[0]
window_size

Timedelta('364 days 00:00:00')

### train table

In [28]:
# default time window sampler used inside make_train_table
time_window_df = rolling_window_sampler(
    ds.min_time, ds.train_max_time, window_size, window_size
)
time_window_df

,window_min_time,window_max_time
0,1996-06-25 00:00:01,1997-06-24 00:00:01
1,1997-06-24 00:00:01,1998-06-23 00:00:01
2,1998-06-23 00:00:01,1999-06-22 00:00:01
3,1999-06-22 00:00:01,2000-06-20 00:00:01
4,2000-06-20 00:00:01,2001-06-19 00:00:01
5,2001-06-19 00:00:01,2002-06-18 00:00:01
6,2002-06-18 00:00:01,2003-06-17 00:00:01
7,2003-06-17 00:00:01,2004-06-15 00:00:01
8,2004-06-15 00:00:01,2005-06-14 00:00:01
9,2005-06-14 00:00:01,2006-06-13 00:00:01


In [29]:
train_table = ds.make_train_table("churn", window_size)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [30]:
# here count is the number of products which contributed to the sum aggregation in ltv
train_table.df

,window_min_time,window_max_time,customer_id,churn
0,1996-06-25 00:00:01,1997-06-24 00:00:01,1843200,True
1,1996-06-25 00:00:01,1997-06-24 00:00:01,1843201,True
2,1996-06-25 00:00:01,1997-06-24 00:00:01,1843202,True
3,1996-06-25 00:00:01,1997-06-24 00:00:01,1843203,True
4,1996-06-25 00:00:01,1997-06-24 00:00:01,1843204,True
...,...,...,...,...
31453276,2012-06-05 00:00:01,2013-06-04 00:00:01,921595,False
31453277,2012-06-05 00:00:01,2013-06-04 00:00:01,921596,False
31453278,2012-06-05 00:00:01,2013-06-04 00:00:01,921597,True
31453279,2012-06-05 00:00:01,2013-06-04 00:00:01,921598,True


In [31]:
# frequency of churn in percent
train_table.df["churn"].value_counts(normalize=True) * 100

churn
True     95.140863
False     4.859137
Name: proportion, dtype: float64

_Note: this is very skewed! Val table is not this skewed (95% 0-counts vs 52%)._

## val table

In [36]:
# time sampler used inside make_val_table
time_window_df = one_window_sampler(ds.train_max_time, window_size)
time_window_df

,window_min_time,window_max_time
0,2014-04-16 00:00:01,2015-04-15


In [37]:
val_table = ds.make_val_table("churn", window_size)

In [38]:
val_table.df

,window_min_time,window_max_time,customer_id,churn
0,2014-04-16 00:00:01,2015-04-15,0,False
1,2014-04-16 00:00:01,2015-04-15,1,True
2,2014-04-16 00:00:01,2015-04-15,2,False
3,2014-04-16 00:00:01,2015-04-15,3,True
4,2014-04-16 00:00:01,2015-04-15,4,False
...,...,...,...,...
1850188,2014-04-16 00:00:01,2015-04-15,1843195,True
1850189,2014-04-16 00:00:01,2015-04-15,1843196,True
1850190,2014-04-16 00:00:01,2015-04-15,1843197,True
1850191,2014-04-16 00:00:01,2015-04-15,1843198,True


In [39]:
val_table.df["churn"].value_counts(normalize=True) * 100

churn
False    52.623321
True     47.376679
Name: proportion, dtype: float64